# Anomaly Detection Using Natural Language Processing

In this project I explored text message data and created models to predict if a message is spam or not.

Here I look at the beginning of the dataset and encode the target labels for spam and not spam.

In [22]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

spam_data = pd.read_csv('spam.csv')
spam_data['target'] = np.where(spam_data['target']=='spam', 1, 0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


Split the data into training and testing sets.

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

Determine whether the dataset is balanced by calculating the percentage of spam messages.

In [6]:
sum_spam = sum(spam_data['target'])
ratio_spam = sum_spam / len(spam_data)
spam_percentage = ratio_spam * 100
spam_percentage
# data is unbalanced, with more instances of non-spam messages

13.406317300789663

Next, I want to determine how important certain words are to spam messages and non-spam messages, including how often they appear in these messages. So I fit and transform the training data using a Tfidf Vectorizer to determine the 20 features with the smallest and largest tf-idf values.

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer().fit(X_train)
X_train_vectorized = vect.transform(X_train)
feature_names = np.array(vect.get_feature_names())
sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
smallest = feature_names[sorted_tfidf_index[:20]]
largest = feature_names[sorted_tfidf_index[:-21:-1]]
vect_data = X_train_vectorized.max(0).toarray()[0]
frame_small = {'value': pd.Series(vect_data[sorted_tfidf_index[:20]]), 'name': pd.Series(smallest)} 
df_small = pd.DataFrame(frame_small).sort_values(['value', 'name'])
frame_large = {'value': pd.Series(vect_data[sorted_tfidf_index[:-21:-1]]), 'name': pd.Series(largest)} 
df_large = pd.DataFrame(frame_large).sort_values(['value', 'name'], ascending=[False, True])
series_small = pd.Series(data = df_small['value'].values, index = df_small['name'])
series_large = pd.Series(data = df_large['value'].values, index = df_large['name'])
w_values = (series_small, series_large)
w_values[0].name = None
w_values[1].name = None
w_values

(name
 aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64,
 name
 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        

Next, I begin some feature-engineering. I determine the average length (number of characters) for spam and not-spam messages.

In [8]:
data = spam_data
data['length'] = [len(x) for x in data['text']]
avg_spam = data['length'].loc[data['target'] == 1].mean()
avg_not_spam = data['length'].loc[data['target'] == 0].mean()
w_length = (avg_not_spam, avg_spam)
w_length

(71.02362694300518, 138.8661311914324)

<br>
The following function helps combine new features into the training data:

In [9]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

Here I calculate the average number of digits for spam and not-spam messages. I plan to use this feature in my final model.

In [10]:
data = spam_data
num_digits = []
for x in data['text']:
    number = sum(y.isdigit() for y in x)
    num_digits.append(number)
data['digit'] = num_digits
digit_spam = data['digit'].loc[data['target'] == 1].mean()
digit_not_spam = data['digit'].loc[data['target'] == 0].mean()
w_digits = (digit_not_spam, digit_spam)
w_digits

(0.2992746113989637, 15.759036144578314)

Another feature I want to engineer and consider is the average number of non-word characters (anything other than a letter, digit, or underscore) per message for spam and not-spam messages.

In [11]:
import re

data = spam_data
num_other = []
for x in data['text']:
    char_values = re.findall(r'\W', x)
    len_char = len(char_values)
    num_other.append(len_char)
data['other'] = num_other
other_spam = data['other'].loc[data['target'] == 1].mean()
other_not_spam = data['other'].loc[data['target'] == 0].mean()
w_char = (other_not_spam, other_spam)
w_char

(17.29181347150259, 29.041499330655956)

### Final Model

Finally, I fit and transform the training data using a Count Vectorizer. I ignore terms that have a document frequency lower than 5 and I use character n-grams from n = 2 to n = 5.

In the Count Vectorizer I specified the "analyzer = 'char_wb'" parameter in order to create character n-grams only from text inside word boundaries. This helps make the model more robust to spelling mistakes.

I used the document-term matrix and the following additional features:
- length of messages (number of characters)
- number of digits per message
- number of non-word characters (anything other than a letter, digit or underscore)

Then, I fit a Logistic Regression model with regularization C = 100. To assess model performance, I computed the area under the curve (AUC) score using the transformed test data. Additionally, I found the 10 smallest and 10 largest coefficients from the model.

In [20]:
def spam_model():
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score
    from sklearn.feature_extraction.text import CountVectorizer

    message_length_train = [len(x) for x in X_train]
    message_length_test = [len(x) for x in X_test]
    num_digits_train = []
    num_other_train = []
    for x in X_train:
        number = sum(y.isdigit() for y in x)
        num_digits_train.append(number)
        char_values = re.findall(r'\W', x)
        len_char = len(char_values)
        num_other_train.append(len_char)
    num_digits_test = []
    num_other_test = []
    for w in X_test:
        number = sum(q.isdigit() for q in w)
        num_digits_test.append(number)
        char_values = re.findall(r'\W', w)
        len_char = len(char_values)
        num_other_test.append(len_char)
    vect = CountVectorizer(min_df = 5, analyzer = 'char_wb', ngram_range = (2,5)).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_train_v_feat = add_feature(X_train_vectorized, [message_length_train, num_digits_train, num_other_train])
    X_test_vectorized = vect.transform(X_test)
    X_test_v_feat = add_feature(X_test_vectorized, [message_length_test, num_digits_test, num_other_test])
    clf = LogisticRegression(C = 100)
    clf.fit(X_train_v_feat, y_train)
    predictions = clf.predict(X_test_v_feat)
    auc_score = roc_auc_score(y_test, predictions)
    feature_names = vect.get_feature_names()
    feature_names.append('length_of_doc')
    feature_names.append('digit_count')
    feature_names.append('non_word_char_count')
    feature_names = np.array(feature_names)
    sorted_coef_index = clf.coef_[0].argsort()
    small = feature_names[sorted_coef_index[:10]]
    large = feature_names[sorted_coef_index[:-11:-1]]

    coef_data = clf.coef_.max(0) #.toarray()[0]
    frame_small = {'value': pd.Series(coef_data[sorted_coef_index[:10]]), 'feature_name': pd.Series(small)} 
    df_small = pd.DataFrame(frame_small).sort_values(['value', 'feature_name'])
    frame_large = {'value': pd.Series(coef_data[sorted_coef_index[:-11:-1]]), 'feature_name': pd.Series(large)} 
    df_large = pd.DataFrame(frame_large).sort_values(['value', 'feature_name'], ascending=[False, True])
    series_small = pd.Series(data = df_small['value'].values, index = df_small['feature_name'])
    series_large = pd.Series(data = df_large['value'].values, index = df_large['feature_name'])
    w_final = (auc_score, series_small, series_large)
    return w_final

In [23]:
spam_model()

(0.9813973821367333,
 feature_name
 ..    -1.356845
 .     -1.141882
  i    -0.854167
  go   -0.721966
  y    -0.718508
 ?     -0.688946
 pe    -0.651174
 go    -0.618091
 h     -0.610952
 ok    -0.601233
 dtype: float64,
 feature_name
 digit_count    1.454480
 co             0.741110
 ne             0.728373
 ww             0.710818
 ar             0.636197
 ia             0.628743
 xt             0.608064
  ch            0.604465
 mob            0.583807
  a             0.575787
 dtype: float64)

This model has an AUC score of 0.98 and can successfully classify messages as either spam or not-spam.